This project aims to detect land cover changes over time using Sentinel-2 satellite imagery. The analysis focuses on identifying urban expansion and deforestation using image segmentation techniques

# **Step 1:** *Collect Sentinel-2 Satellite Images ad preprocessing*

In [19]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!apt-get install git


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git is already the newest version (1:2.34.1-1ubuntu1.12).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.


In [ ]:
!git config --global user.name "hanan99"
!git config --global user.email "hananhabibi1999@gmail.com"


In [ ]:
!git clone https://github.com/hanan99/SeSentinel-2_satellite_analysis.git


Cloning into 'SeSentinel-2_satellite_analysis'...


In [18]:
!cp /content/drive/MyDrive/Colab\ Notebooks/Sentinel-2_satellite_analysis/Sentinel-2_satellite_analysis_veneto.ipynb


total 2454
-rw------- 1 root root 2512678 Feb 22 20:25 Sentinel-2_satellite_analysis_veneto.ipynb


In [ ]:
!mv /Sentinel-2_satellite_analysis_veneto.ipynb /content/SeSentinel-2_satellite_analysis/


mv: cannot stat '/Sentinel-2_satellite_analysis_veneto.ipynb': No such file or directory


In [ ]:
!pip install earthengine-api geemap
!pip install geemap



In [ ]:
import ee
import geemap

In [ ]:
# Initialize the Earth Engine module
ee.Authenticate()
ee.Initialize(project='sound-essence-295918')
print("Google Earth Engine is successfully initialized!")



Google Earth Engine is successfully initialized!


In [ ]:
# Define the bounding box for Veneto
veneto_bbox = ee.Geometry.Rectangle([10.9399, 44.7739, 12.6648, 46.5682])

In [ ]:
# Center the map on the ROI
Map = geemap.Map()
Map.centerObject(veneto_bbox, 8)  # Zoom level 8 for regional view

In [ ]:
# Add the ROI to the map
Map.addLayer(veneto_bbox, {'color': 'red'}, 'Veneto Region')
Map

Map(center=[45.669499615461646, 11.80234999999995], controls=(WidgetControl(options=['position', 'transparent_…

In [ ]:
# Define the time range for the satellite images
start_date = '2020-01-01'
end_date = '2020-12-31'

# Load Sentinel-2 Image Collection for the Veneto region
sentinel2 = ee.ImageCollection('COPERNICUS/S2') \
    .filterBounds(veneto_bbox) \
    .filterDate(start_date, end_date) \
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 10))  # Keep images with <10% cloud cover

# Compute the median image to reduce cloud effects
median_image = sentinel2.median().clip(veneto_bbox)

# Print image information
print(median_image.getInfo())




{'type': 'Image', 'bands': [{'id': 'B1', 'data_type': {'type': 'PixelType', 'precision': 'double', 'min': 0, 'max': 65535}, 'dimensions': [4, 4], 'origin': [10, 44], 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}, {'id': 'B2', 'data_type': {'type': 'PixelType', 'precision': 'double', 'min': 0, 'max': 65535}, 'dimensions': [4, 4], 'origin': [10, 44], 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}, {'id': 'B3', 'data_type': {'type': 'PixelType', 'precision': 'double', 'min': 0, 'max': 65535}, 'dimensions': [4, 4], 'origin': [10, 44], 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}, {'id': 'B4', 'data_type': {'type': 'PixelType', 'precision': 'double', 'min': 0, 'max': 65535}, 'dimensions': [4, 4], 'origin': [10, 44], 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}, {'id': 'B5', 'data_type': {'type': 'PixelType', 'precision': 'double', 'min': 0, 'max': 65535}, 'dimensions': [4, 4], 'origin': [10, 44], 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 

In [ ]:
# Visualization parameters
vis_params = {
    'bands': ['B4', 'B3', 'B2'],  # Red-Green-Blue bands for natural color
    'min': 0,
    'max': 3000,
    'gamma': 1.4
}

# Add Sentinel-2 image to the interactive map
Map.addLayer(median_image, vis_params, 'Sentinel-2 Image')
Map


Map(center=[45.669499615461646, 11.80234999999995], controls=(WidgetControl(options=['position', 'transparent_…

In [ ]:
# Define the time range for 2015
start_date_2015 = '2015-07-01'  # Sentinel-2 became operational in mid-2015
end_date_2015 = '2015-12-31'

# Load Sentinel-2 images for 2015
sentinel2_2015 = ee.ImageCollection('COPERNICUS/S2') \
    .filterBounds(veneto_bbox) \
    .filterDate(start_date_2015, end_date_2015) \
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 10))  # Remove cloudy images

# Compute the median image for 2015
median_image_2015 = sentinel2_2015.median().clip(veneto_bbox)

# Define visualization parameters for true-color images
vis_params = {
    'bands': ['B4', 'B3', 'B2'],  # Red-Green-Blue for natural color
    'min': 0,
    'max': 3000,
    'gamma': 1.4
}

# Add 2015 and 2020 images to the map
Map.addLayer(median_image_2015, vis_params, 'Sentinel-2 2015')
Map.addLayer(median_image, vis_params, 'Sentinel-2 2020')

# Show the map
Map


Map(center=[45.669499615461646, 11.80234999999995], controls=(WidgetControl(options=['position', 'transparent_…

# **Step 2:** *Apply Image Segmentation (Change Detection)*

In the following map urban expansion and deforestation can be observed clearly in red-band differences. Red reflectance is sensitive to vegetation and soil changes.

> Positive values → Increase in red reflectance (e.g., new buildings, urban expansion).


---


> Negative values → Decrease in red reflectance (e.g., vegetation loss, deforestation).
 ---
> Near zero → No significant change.




In [ ]:
# Compute the difference between 2020 and 2015 (Red Band)
change_detection = median_image.select('B4').subtract(median_image_2015.select('B4'))

# Visualization parameters (Highlighting changes)
change_vis_params = {
    'min': -500,
    'max': 500,
    'palette': ['blue', 'white', 'red']  # Blue = decrease, White = no change, Red = increase
}

# Add change detection layer to the map
Map.addLayer(change_detection, change_vis_params, 'Change Detection (2015-2020)')
Map


Map(center=[45.669499615461646, 11.80234999999995], controls=(WidgetControl(options=['position', 'transparent_…

# **Step 5**: *Land Cover Classification*

Manually define example points in the region that represent different land types.

In [ ]:
# Define sample points for Urban Areas
urban = ee.FeatureCollection([
    ee.Feature(ee.Geometry.Point([11.877, 45.403]), {'class': 0}),
    ee.Feature(ee.Geometry.Point([11.883, 45.407]), {'class': 0}),
])  # Class 0 = Urban

# Define sample points for Vegetation
vegetation = ee.FeatureCollection([
    ee.Feature(ee.Geometry.Point([11.937, 45.326]), {'class': 1}),
    ee.Feature(ee.Geometry.Point([11.943, 45.321]), {'class': 1}),
])  # Class 1 = Vegetation

# Define sample points for Water Bodies
water = ee.FeatureCollection([
    ee.Feature(ee.Geometry.Point([12.301, 45.432]), {'class': 2}),
    ee.Feature(ee.Geometry.Point([12.311, 45.429]), {'class': 2}),
])  # Class 2 = Water

# Define sample points for Barren Land
barren = ee.FeatureCollection([
    ee.Feature(ee.Geometry.Point([11.735, 45.275]), {'class': 3}),
    ee.Feature(ee.Geometry.Point([11.740, 45.270]), {'class': 3}),
])  # Class 3 = Barren Land

# Merge all classes into one dataset
training_samples = urban.merge(vegetation).merge(water).merge(barren)


**Extract Features from Sentinel-2**

In [ ]:
# Select Sentinel-2 spectral bands relevant for classification
bands = ['B2', 'B3', 'B4', 'B8', 'B11', 'B12']  # Blue, Green, Red, NIR, SWIR1, SWIR2

# Sample training data from Sentinel-2 image
training_data = median_image.select(bands).sampleRegions(
    collection=training_samples,
    properties=['class'],
    scale=30  # Resolution in meters
)

# Print the number of training samples
print("Number of training samples:", training_data.size().getInfo())


Number of training samples: 8


**Train a Machine Learning Model (Random Forest)**

In [ ]:
# Train the classifier using Random Forest with 50 trees
classifier = ee.Classifier.smileRandomForest(50).train(
    features=training_data,
    classProperty='class',
    inputProperties=bands
)

# Apply classification to the entire Veneto region
classified_image = median_image.select(bands).classify(classifier)


# **Step 5:** *Visualize and Compare Changes*

**Visualize the Classified Land Cover Map**

In [ ]:
# Define visualization parameters
class_vis = {
    'min': 0,
    'max': 3,
    'palette': ['red', 'green', 'blue', 'gray']  # Urban, Vegetation, Water, Barren
}

# Add classification result to the map
Map.addLayer(classified_image, class_vis, 'Land Cover Classification')
Map


Map(center=[45.669499615461646, 11.80234999999995], controls=(WidgetControl(options=['position', 'transparent_…

In [ ]:

# Load Sentinel-2 images for 2015
sentinel2_2015 = ee.ImageCollection('COPERNICUS/S2') \
    .filterBounds(veneto_bbox) \
    .filterDate('2015-07-01', '2015-12-31') \
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20)) \
    .limit(10)
    # Allow 20% cloud cover and limits to 10 images to reduce memory usage

# Count the number of available images
image_count_2015 = sentinel2_2015.size().getInfo()
print("Number of Sentinel-2 images in 2015:", image_count_2015)


Number of Sentinel-2 images in 2015: 10


This is a classified land cover map for 2015

In [ ]:
# Apply classification model to 2015 image
classified_image_2015 = median_image_2015.select(bands).classify(classifier)

# Visualize 2015 land cover classification
Map.addLayer(classified_image_2015, class_vis, 'Land Cover 2015')
Map


Map(bottom=94563.0, center=[45.51019654498558, 12.1673583984375], controls=(WidgetControl(options=['position',…

**How to Interpret The following map:**

> 🔴 Red = Urban Expansion (newly developed areas)
---
> 🟢 Green = Vegetation Increase (forestation, regrowth)
---
> 🔵 Blue = Water Body Changes (drying lakes/rivers or new water bodies)
---
> ⚪ Gray = No Change


In [ ]:
# Compute land cover changes
land_change = classified_image.subtract(classified_image_2015)

# Define visualization for changes
change_vis_params = {
    'min': -1,
    'max': 1,
    'palette': ['red', 'green', 'blue']  # Urban, Vegetation, Water
}

# Add land cover changes to the map
Map.addLayer(land_change, change_vis_params, 'Land Cover Changes(2015-2020)')

Map

Map(bottom=23891.0, center=[44.939529212272305, 12.431152614561771], controls=(WidgetControl(options=['positio…

**Quantify the Changes**\
This calculates the total area (in square meters) for each land cover change